### קוד מבוא

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import os
import sys

import folium
import fiona

from shapely.geometry import Point


PROJ: proj_create_from_database: SQLite error on SELECT key, value FROM metadata WHERE key IN ('DATABASE.LAYOUT.VERSION.MAJOR', 'DATABASE.LAYOUT.VERSION.MINOR'): no such table: metadata


In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [4]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [5]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [6]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [7]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [8]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


### העלת קבצים

In [9]:
path = os.getcwd()

software_root_folder = os.path.dirname(path)


In [10]:

software_folder_location = r'{}\create_potential_aprt'.format(software_root_folder)


In [11]:


df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_folder_location))


##### לייצר יח"ד קיים פר אזור תנועה

In [12]:
forecast_2020=up_load_df(r'{}\background_files'.format(software_folder_location),'2020_jtmt_forcast_full_240226')

In [13]:
x=r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_folder_location)

In [14]:
taz=up_load_shp(x)

c:\Users\eliyal\AppData\Local\anaconda3\envs\create_forecast_v9.0.0\lib\site-packages\pyogrio\core.py:24: RuntimeWarning: Could not detect PROJ data files.  Set PROJ_LIB environment variable to the correct path.
  _init_proj_data()


In [15]:
taz=pd.merge(taz,forecast_2020[['Taz_num','aprt_20']].query('aprt_20>0'),on='Taz_num')

In [16]:
path=r'{}\background_files\BNTL_2022.gdb'.format(software_folder_location)
bld=up_load_gdb(path,'BLDG_Clip')
bld_poi=up_load_gdb(path,'POI_BLDG_Clip')
bld=bld.merge(bld_poi[[ 'FCODE', 'USG_GROUP', 'USG_CODE','USG_SP_NM_LTN','BLDG_ID']],how='left',left_on='UNIQ_ID',right_on='BLDG_ID')
bld=bld.loc[bld['FCODE_y'].isna()]
bld['bld_area']=bld.area
bld_point=make_point(bld).fillna(0)
bld_point.loc[bld_point['HEIGHT']<=0,'HEIGHT']=bld_point['HI_PNT_Z']-bld_point['HT_LAND']
ceiling_height=3
bld_point['bld_m3']=(bld_point['HEIGHT']/ceiling_height).astype(int)*bld_point['bld_area'].astype(int)
bld_point.loc[(bld_point['HEIGHT']-ceiling_height)<=0,'bld_m3']=bld_point['bld_area']
bld_point=gpd.sjoin(bld_point,taz)
col=['bld_m3','centroid','Taz_num','aprt_20']
bld_point=bld_point[col]
bld_point['bld_m3']=bld_point['bld_m3'].astype(int)
sum_bld_m3=pd.pivot_table(bld_point.drop(columns='centroid'),index='Taz_num',aggfunc=sum)[['bld_m3']].reset_index().rename(columns={'bld_m3':'bld_m3_taz'})
bld_point=bld_point.merge(sum_bld_m3,on='Taz_num')
bld_point['aprt_in_bld_from_pre_of_taz']=bld_point['bld_m3']/bld_point['bld_m3_taz']*bld_point['aprt_20']
bld_point['aprt_in_bld_from_pre_of_taz']=bld_point['aprt_in_bld_from_pre_of_taz'].astype(int)


C:\Users\eliyal\AppData\Local\Temp\ipykernel_17852\3669232119.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bld_point=make_point(bld).fillna(0)
C:\Users\eliyal\AppData\Local\Temp\ipykernel_17852\3669232119.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 31.3225      20.58816099  23.03316663 ...  79.56155771  86.00171698
 411.10230836]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  bld_point.loc[(bld_point['HEIGHT']-ceiling_height)<=0,'bld_m3']=bld_point['bld_area']
C:\Users\eliyal\AppData\Local\Temp\ipykernel_17852\3669232119.py:16: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. 

עד כאן יש לי שכבה של כמות יח"ד בכל בניין

עכשיו צריך להעלות את המקורות מידע שלנו

תכניות אינדקס

In [17]:
client_data_folder_location=df_inputs_outputs['location'][0]

In [18]:
path_to_index_df=r'{}\For_approval\Reference_tabels'.format(client_data_folder_location)

In [19]:
name_of_index_file=r'index_format_for_creating_forecast_jtmt_input_{}_{}'.format(df_inputs_outputs['location'][1],df_inputs_outputs['location'][2])

השורה הבא מועדת לפורענות

name_of_index_file=r'‏‏index_format_for_creating_forecast_jtmt_input_with_project_240923'

In [20]:
plans_df=up_load_df(path_to_index_df,name_of_index_file)

c:\Users\eliyal\AppData\Local\anaconda3\envs\create_forecast_v9.0.0\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [21]:
plans_shp=up_load_shp(r'{}\For_approval\Reference_tabels\shp\gvul_index.shp'.format(client_data_folder_location))

להעלות את שכבת הרקל

In [22]:
rakal=up_load_gdb(r'{}\For_approval\Reference_tabels\shp\tochnit_check.gdb'.format(client_data_folder_location),'lrt_policy')

להוריד אזורים שכבר יש תכניות באינדקס

In [23]:
plans_shp['type']='plans'

In [24]:
col=['type','geometry']
rakal=rakal.overlay(plans_shp[col], how='union').query('type.isna()')

לחלק לפי אזורי תנועה ורק אז לייצר את הקיבולת

להעלות את שכבת האזורי תנועה הנדרשים לפרוייקט

In [25]:
taz_for_proj=up_load_gdb(r'{}\For_approval\Reference_tabels\shp\tochnit_check.gdb'.format(client_data_folder_location),'TAZ_211028_V3_Published_with_client_changes')

c:\Users\eliyal\AppData\Local\anaconda3\envs\create_forecast_v9.0.0\lib\site-packages\pyogrio\raw.py:194: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiPolygon' is converted to 'MultiPolygon Z'
  result = ogr_read(


In [26]:
taz_for_proj['taz']=1

In [27]:
rakal['rakal']=1

In [28]:
rakal=rakal.overlay(taz_for_proj[['taz','geometry']], how='union').query('rakal==1 & taz==1')

In [29]:
rakal['area']=rakal.area

לייצר את תופסת יח"ד של רקל

In [30]:
aprt_size=130

In [31]:
rakal['potential_aprt']=rakal['area']*7.2*0.5/aprt_size

In [32]:
#כמה יח"ד יש בכל אחד מהפוליגונים?
rakal['id']=rakal.index

In [33]:
bld_point_for_plans_need_add_aprt=gpd.sjoin(bld_point,rakal)

In [34]:
bld_point_for_plans_need_add_aprt=bld_point_for_plans_need_add_aprt.drop(columns='centroid').pivot_table(index='id',aggfunc=sum)[['aprt_in_bld_from_pre_of_taz']]

C:\Users\eliyal\AppData\Local\Temp\ipykernel_17852\961791580.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  bld_point_for_plans_need_add_aprt=bld_point_for_plans_need_add_aprt.drop(columns='centroid').pivot_table(index='id',aggfunc=sum)[['aprt_in_bld_from_pre_of_taz']]


In [35]:
rakal=rakal.set_index('id')

In [36]:
rakal['aprt_20']=bld_point_for_plans_need_add_aprt['aprt_in_bld_from_pre_of_taz']

In [37]:
rakal=rakal.fillna(0)

C:\Users\eliyal\AppData\Local\Temp\ipykernel_17852\3539812066.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rakal=rakal.fillna(0)


In [38]:
rakal['add_potential_aprt']=rakal['potential_aprt']-rakal['aprt_20']

In [39]:
rakal=rakal.loc[rakal['add_potential_aprt']>50]

לייצר שימושים תעסוקה ברקל

In [40]:
rakal['Business_m2']=rakal['area']*7.2*0.1

In [41]:
rakal['Commerce_m2']=rakal['area']*7.2*0.1

להעלות את שכבת בינוי חדש


In [42]:
new_bld=up_load_gdb(r'{}\For_approval\Reference_tabels\shp\tochnit_check.gdb'.format(client_data_folder_location),'new_bld')

In [43]:
new_bld['add_potential_aprt']=new_bld['Shape_Area']*new_bld['prec_for_aprt']*new_bld['neto_dens']/1000

In [44]:
new_bld=new_bld.loc[new_bld['add_potential_aprt']>50]

In [45]:
new_bld['type']='new_bld'

להעלות את שכבת מכפיל

In [46]:
machpil=up_load_gdb(r'{}\For_approval\Reference_tabels\shp\tochnit_check.gdb'.format(client_data_folder_location),'urban_density')

עכשיו צריך לחורר משכבת מכפיל את הפוליגונים של רקל ושל תכניות

In [47]:
rakal['type']='lrt'

In [48]:
col=['type','geometry']

In [49]:
machpil=machpil.overlay(pd.concat([rakal[col],plans_shp[col],new_bld[col]]), how='union').query('type.isna()')

In [50]:
machpil['machpil']=1

In [51]:
machpil=machpil.overlay(taz_for_proj[['taz','geometry']], how='union').query('machpil==1')

c:\Users\eliyal\AppData\Local\anaconda3\envs\create_forecast_v9.0.0\lib\site-packages\geopandas\geodataframe.py:2675: UserWarning: `keep_geom_type=True` in overlay resulted in 26 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


לייצר כמות יח"ד בשביל הכפלה

In [52]:
#כמה יח"ד יש בכל אחד מהפוליגונים?
machpil['id']=machpil.index

In [53]:
bld_point_for_plans_need_add_aprt=gpd.sjoin(bld_point,machpil)

In [54]:
bld_point_for_plans_need_add_aprt=bld_point_for_plans_need_add_aprt.drop(columns='centroid').pivot_table(index='id',aggfunc=sum)[['aprt_in_bld_from_pre_of_taz']]

C:\Users\eliyal\AppData\Local\Temp\ipykernel_17852\961791580.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  bld_point_for_plans_need_add_aprt=bld_point_for_plans_need_add_aprt.drop(columns='centroid').pivot_table(index='id',aggfunc=sum)[['aprt_in_bld_from_pre_of_taz']]


In [55]:
machpil=machpil.set_index('id')

In [56]:
machpil['aprt_20']=bld_point_for_plans_need_add_aprt['aprt_in_bld_from_pre_of_taz']

In [57]:
machpil=machpil.fillna(0)

C:\Users\eliyal\AppData\Local\Temp\ipykernel_17852\3136594500.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  machpil=machpil.fillna(0)


In [58]:
machpil['add_potential_aprt']=machpil['aprt_20']*machpil['coefficient']

In [59]:
machpil=machpil.loc[machpil['add_potential_aprt']>25]

עכשיו שיש לי תופסת יח"ד ברקל בינוי חדש ומכפיל צריך לצרף את זה לאינדקס השכבה ולאקסל

In [60]:
plans_shp.drop(columns='type',inplace=True)

In [61]:
max_id =int(plans_shp['id'].max())
num_new_rows = int(len(machpil))

machpil.reset_index(inplace=True)

In [62]:
machpil['id'] = range(max_id + 1, max_id + 1 + num_new_rows)

In [63]:
machpil = machpil.rename(columns={'add_potential_aprt': 'add_aprt'})

In [64]:
machpil['plan_name']='מכפיל'

In [65]:
machpil['status']='פוטנציאל'

In [66]:
machpil['emp_fill_factor']=1

In [67]:
machpil['Risk_factor']=1

In [68]:
joined_plans_shp=pd.concat([plans_shp,machpil[['id','geometry']]])
joined_plans_df=pd.concat([plans_df,machpil[['id','add_aprt','plan_name','status','Risk_factor','emp_fill_factor']]])


הוספה של רקל לשכבה וטבלאות

In [69]:
max_id =int(joined_plans_shp['id'].max())
num_new_rows = int(len(rakal))


In [70]:
rakal.reset_index(inplace=True)

In [71]:
rakal['id'] = range(max_id + 1, max_id + 1 + num_new_rows)

In [72]:
rakal = rakal.rename(columns={'add_potential_aprt': 'add_aprt'})

In [73]:
rakal['plan_name']='רקל'

In [74]:
rakal['status']='פוטנציאל'

In [75]:
rakal['emp_fill_factor']=1

In [76]:
rakal['Risk_factor']=1

In [77]:
joined_plans_shp=pd.concat([joined_plans_shp,rakal[['id','geometry']]])
joined_plans_df=pd.concat([joined_plans_df,rakal[['id','plan_name','status','add_aprt','Business_m2','Commerce_m2','Risk_factor','emp_fill_factor']]])


הוספה של בינוי חדש לשכבה וטבלאות

In [78]:
max_id =int(joined_plans_shp['id'].max())
num_new_rows = int(len(new_bld))


In [79]:
new_bld.reset_index(inplace=True)

In [80]:
new_bld['id'] = range(max_id + 1, max_id + 1 + num_new_rows)

In [81]:
new_bld = new_bld.rename(columns={'add_potential_aprt': 'add_aprt'})

In [82]:
new_bld['plan_name']='בינוי חדש'

In [83]:
new_bld['status']='פוטנציאל'

In [84]:
new_bld['emp_fill_factor']=1

In [85]:
new_bld['Risk_factor']=1

In [86]:
joined_plans_shp=pd.concat([joined_plans_shp,new_bld[['id','geometry']]])
joined_plans_df=pd.concat([joined_plans_df,new_bld[['id','plan_name','status','add_aprt','Risk_factor','emp_fill_factor']]])


הוספה של מימושים

In [87]:
mimoshim_jtmt=up_load_df(r'W:\Data\Forecast\Tools\create_forecast_ad_hoc\create_potential_aprt','mimoshim_jtmt')

In [88]:
def update_f_value(df, status_value, mimoshim_df, column_name,x):
    df.loc[(df['status'] == status_value) & (df['add_aprt'] <= 250), 'F{}'.format(column_name)] = mimoshim_df.iloc[x][column_name]
    df.loc[(df['status'] == status_value) & (df['add_aprt'] > 250) & (df['add_aprt'] <= 500), 'F{}'.format(column_name)] = mimoshim_df.iloc[x+1][column_name]
    df.loc[(df['status'] == status_value) & (df['add_aprt'] > 500), 'F{}'.format(column_name)] = mimoshim_df.iloc[x+2][column_name]
    df.loc[(df['status'] == status_value) & (df['add_aprt'] ==0), 'F{}'.format(column_name)] = mimoshim_df.iloc[x+1][column_name]

    return df


In [89]:
stat_lst=['מאושר','הליכים','פרה_רולינג','פוטנציאל','היתר']

In [90]:
col_lst=['2030','2035','2040']

In [91]:
x=0
for i in stat_lst:

    for c in col_lst:
        joined_plans_df=update_f_value(joined_plans_df, i, mimoshim_jtmt, c,x)
    
    print(x,'-',i)
    
    x=x+3

0 - מאושר
3 - הליכים
6 - פרה_רולינג
9 - פוטנציאל
12 - היתר


In [92]:
col=['F2025',
 'F2030',
 'F2035',
 'F2040']

In [93]:
joined_plans_df['add_aprt_till_2040']=joined_plans_df['add_aprt']*joined_plans_df[col].sum(axis=1)

In [94]:
joined_plans_shp=joined_plans_shp.merge(joined_plans_df[['id','status']],on='id',how='left')

In [95]:
joined_plans_shp = joined_plans_shp.set_crs("EPSG:2039", allow_override=True)
print(joined_plans_shp.crs)

EPSG:2039


In [97]:
joined_plans_shp.to_file(r'{}\For_approval\Reference_tabels\shp\gvul_index_with_ponten.shp'.format(client_data_folder_location),encoding='utf-8', engine = 'fiona')

In [ ]:
print(joined_plans_shp.crs)

In [98]:
joined_plans_df.to_excel(r'{}\For_approval\Reference_tabels\index_format_for_creating_forecast_jtmt_input_{}_{}_with_poten_jtmt.xlsx'.format(client_data_folder_location,df_inputs_outputs['location'][1],df_inputs_outputs['location'][2]),index=False)

col=['status','add_aprt','add_aprt_till_2040']
df_sum_jtmt_mimosh_by_status=pd.pivot_table(joined_plans_df[col],index='status',aggfunc=sum)
הוספה של מימושים של העירייה
mimoshim_jeru=up_load_df(r'W:\Data\Forecast\Tools\create_potential_aprt','‏‏mimoshim_jeru')
x=0
for i in stat_lst:

    for c in col_lst:
        joined_plans_df=update_f_value(joined_plans_df, i, mimoshim_jeru, c,x)
    
    print(x,'-',i)
    
    x=x+3
col=['F2025',
 'F2030',
 'F2035',
 'F2040']
joined_plans_df['add_aprt_till_2040']=joined_plans_df['add_aprt']*joined_plans_df[col].sum(axis=1)
joined_plans_df.to_excel(r'{}\For_approval\Reference_tabels\index_format_for_creating_forecast_jtmt_input_{}_{}_with_poten_muni.xlsx'.format(client_data_folder_location,df_inputs_outputs['location'][1],df_inputs_outputs['location'][2]))
col=['status','add_aprt','add_aprt_till_2040']
df_sum_muni_mimosh_by_status=pd.pivot_table(joined_plans_df[col],index='status',aggfunc=sum)
df=pd.merge(df_sum_muni_mimosh_by_status,df_sum_jtmt_mimosh_by_status[['add_aprt_till_2040']],left_index=True,right_index=True,suffixes=('_muni','_jtmt'))
# חישוב סכום עבור עמודה מסוימת (לדוגמה, עמודה A)
sum_row = pd.DataFrame(df.sum()).T


# הוספת כותרת לשורת הסיכום (אם יש צורך)
sum_row['status'] = 'Total'
sum_row.set_index('status',inplace=True)

# הוספת שורת הסיכום ל-DataFrame המקורי

df_with_sum = pd.concat([df, sum_row], ignore_index=False)
df_with_sum.to_excel(r'{}\For_approval\Reference_tabels\sum_by_status_{}_{}.xlsx'.format(client_data_folder_location,df_inputs_outputs['location'][1],df_inputs_outputs['location'][2]))